## UN Solo Project
In this project, you'll be doing some exploratory analysis on two country-level metrics, gross domestic product (GDP) per capita and overall life expectancy. After completing the guided practice section, you will have a chance to find some additional data and do some more exploring of your own.

In [1]:
import pandas as pd  
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
gdp = pd.read_csv('../data/gdp_per_capita.csv')
gdp.rename(columns={"Country or Area": "Country",'Value':'GDP'}, inplace=True)
gdp.drop(columns=['Value Footnotes'], axis = 1, inplace=True)
gdp.head()

,Country,Year,GDP
0,Afghanistan,2020,1970.560169
1,Afghanistan,2019,2065.036235
2,Afghanistan,2018,2033.804389
3,Afghanistan,2017,2058.400221
4,Afghanistan,2016,2057.067978
